# 클래스 생성

In [9]:
class Node: # node 에 대한 정의, 정보
    
    def __init__(self, val, id_=None, left=None, right=None):
        self.val = val
        self.id_ = id_
        self.left = left
        self.right = right
    
    def __repr__(self):
        return self.val
    
    def show(self):
        print('val=', self.val)
        if self.left:
            print('left=',self.left)
            self.left.show()
            
        if self.right:
            print('right=',self.right)
            self.right.show()
        
        print('--------------------------------')

        
    def insert(self, node):
        
        if not self.right: 
            self.right = node
        
        elif isinstance(self.right, Nonterminal) and (self.right.left == None or self.right.right == None): # left 가 연산자이면
            self.right.insert(node) # left 에 insert 함수 실행
        
        elif not self.left: # right 비어있으면
            self.left = node # right 에 넣고
        
        elif isinstance(self.left, Nonterminal): # right 가 연산자이면
            self.left.insert(node) # right 에 insert 함수 실행
            
        
        
class Terminal(Node): #숫자
    
    def __init__(self, val, id_=None):
        super().__init__(val, id_)
        #self.node_val = float(self.node_val) # 실수 타입
    
    # terminal 인지 아닌지 판별하는기능  
    def calculate(self):
        if type(self.val)==float or type(self.val)==int:
            return True
        else:
            return False
        
        
class Nonterminal(Node): #연산자 , 차일드 관리
    
    def __init__(self, val, id_=None,  left=None, right=None):
        super().__init__(val, id_, left, right)
        #self.node_val = str(self.node_val)
    
    def calculate(self): 
        if type(self.val)==str:
            return True            
            
        else:
            return False
        

# 실행 함수 

In [53]:
# ====================================
# infix -> postfix
# ====================================
import re
prec = {'*': 3, '/': 3,
        '+': 2, '-': 2,
        '(': 1 }

def tokenize(expStr):
    RE = re.compile(r'(?:(?<=[^\d\.])(?=\d)|(?=[^\d\.]))', re.MULTILINE)
    result = [x for x in re.sub(RE, " ", expStr).split(" ") if x]
    
    if result[0]=='-' and is_digit(result[1]):#result[1].isdigit():
        tmp = -float(result[1])
        result.pop(0)
        result[0] = str(tmp)
        
    return result

def is_digit(str):
    try:
        tmp = float(str)
        return True
    except ValueError:
        return False


def infix_to_postfix(S):
    stack = []
    postfix = []
   
    for c in S: 
        if c == '(': # 여는괄호
            stack.append(c)      
        
        elif c == ')': # 닫는괄호가 나오면
            while stack[-1] != '(': # 여는괄호 나오기 전까지 연산자를 postfix에 출력
                postfix.append(stack.pop())
            stack.pop()
        
        else: # 나머지
            if c in prec: # operator 또는 ( 일때
                if len(stack)==0: # 스택 비어있으면 
                    stack.append(c) # 스택에 추가
                elif prec[stack[-1]] >= prec[c]: # 스택 마지막 요소의 연산자 우선순위가 현재 요소 우선순위보다 크거나같다면
                    while len(stack)!=0 and prec[stack[-1]] >= prec[c]: # 스택이 비어있지 않고, 현재의 우선순위가 스택의 마지막꺼보다 더 낮은 동안
                        postfix.append(stack.pop()) # 스택의 마지막 연산자 빼서 answer 에 추가
                    stack.append(c) 
                else:
                    stack.append(c) # 스택에 추가            
                    
            else: # 피연산자일때
                postfix.append(c)
    
    while len(stack)!=0 : # 스택에 남아있는 연산자를 postfix에 추가
        postfix.append(stack.pop())
    
    return postfix


def isOperator(string):
    if string in ['+', '-', '*', '/']:
        return True
    return False



# ====================================
# postfix -> tree
# ====================================
def postfix_to_tree(postfix):
    
    root = Node(postfix[-1])
    for i in reversed(postfix[:-1]):

        if isOperator(i):
            root.insert(Nonterminal(i))
        else:
            root.insert(Terminal(i))
    
    return root


# ====================================
# tree -> calculate
# ====================================

def operator(num1, num2, oper):
    if oper == "+":
        return num1 + num2
    elif oper == "-":
        return num1 - num2
    elif oper == "*":
        return num1 * num2
    else:
        return num1 / num2
  
    
def tree_calculate(node):
    
    if type(node.val)==str and node.left!=None and node.right!=None:
        print(node.val, node.left, node.right)
        left = tree_calculate(node.left)
        right = tree_calculate(node.right)
        node.val = operator(float(left), float(right), node.val)
               
    return node.val

### usage

In [59]:
# infix -> postfix
# 5*(2+3)+(4-5)
# 123+23-0.3*8

infix = input()
infix_tok = tokenize(infix)
pf = infix_to_postfix(infix_tok)
print('postfix = ',pf)


# postfix -> tree
root_node = postfix_to_tree(pf)

# tree -> calculate
tree_calculate(root_node)

-0.3+10
postfix =  ['-0.3', '10', '+']
+ -0.3 10


9.7